Sreejoni Roy
**Reading the Parquet Files**

In this step, I loaded the two cleaned datasets into Python using pd.read_parquet().

df_items contains all the item feature information.

df_users contains the user transaction records.

After loading, I printed the column types and the first few rows from each dataset to quickly check the structure and make sure everything was read correctly.

In [1]:
#Reading the parquet file for features and transcations


import pandas as pd

df_items = pd.read_parquet("Tao Yin_Item_features.parquet")
df_users = pd.read_parquet("TaoYin_User_Transactions_v2.parquet")

print("ITEM FEATURES COLUMNS:\n")
print(df_items.dtypes)
print("\nSample rows from item features:\n")
print(df_items.head())

print("\n\nUSER TRANSACTIONS COLUMNS:\n")
print(df_users.dtypes)
print("\nSample rows from user transactions:\n")
print(df_users.head())


ITEM FEATURES COLUMNS:

Articlenr                  int64
EF000001                 float64
EF000002                 float64
EF000003                 float64
EF000004                 float64
                          ...   
EFUK0017                 float64
EFUK0019                 float64
ETIM                     float64
statistiek_hoofdgroep     object
statistiek_subgroep       object
Length: 4028, dtype: object

Sample rows from item features:

   Articlenr  EF000001  EF000002  EF000003  EF000004  EF000005  EF000006  \
0   34581962       6.0       NaN       NaN       NaN       NaN       NaN   
1   35764300       NaN       NaN       NaN       NaN       NaN       NaN   
2   35823358       NaN       NaN       NaN       NaN       NaN       NaN   
3   28405226       NaN       NaN       NaN       NaN       NaN       NaN   
4   35622440       NaN       NaN       NaN       NaN       NaN       NaN   

   EF000007  EF000008  EF000010  ...  EFDE0031  EFDE0032  EFFR0001  EFFR0002  \
0       NaN   

**Understanding the Output**

The item features table has 4028 columns, mostly numeric feature codes (like EF000001, EFDE0031, etc.). The sample rows show that many of these feature values are missing (NaN), which is expected because not every item has every feature. The last columns (statistiek_hoofdgroep and statistiek_subgroep) are categorical.

The user transactions table looks much cleaner. It includes the basic transaction info such as customer ID, invoice, date, product number, volume, and the two price columns (sum1 and sum2). The sample rows help confirm the structure: repeated invoices, proper date format, and the sales channel as a categorical variable.

**Cleaning the Item Features Dataset**

In this section, I loaded the raw item features parquet file and prepared it for analysis. I first inspected the structure (shape, columns, sample rows) and made sure the product key Articlenr exists. Then, I created a lowercase version of the key (articlenr) so it would match the transactions table later.
The main goal here was to remove duplicates, fix data types, handle missing values, drop useless columns, and reorganise everything into a clean, analysis-ready dataset.

In [2]:
import pandas as pd              # pandas for dataframes
import numpy as np               # numpy for numeric operations
from pathlib import Path         # Path for safe file paths


In [3]:
# setting the folder where my files are located 
DATA_PATH = Path(".")

# The raw file and the cleaned output file
RAW_FEATURES = DATA_PATH / "Tao Yin_Item_features.parquet"          # raw features file (input)
CLEAN_FEATURES = DATA_PATH / "item_features_clean.parquet"          # cleaned features file (output)

In [4]:
features = pd.read_parquet(RAW_FEATURES)                            # loading the parquet into a DataFrame called 'features'

print("Shape (rows, cols):", features.shape)                        # checking the size of rows and columns
print("\nFirst 20 columns:", list(features.columns[:20]))           # checking column names
print("\nPreview rows:\n", features.head(3))                        # checking rows


Shape (rows, cols): (168939, 4028)

First 20 columns: ['Articlenr', 'EF000001', 'EF000002', 'EF000003', 'EF000004', 'EF000005', 'EF000006', 'EF000007', 'EF000008', 'EF000010', 'EF000012', 'EF000013', 'EF000014', 'EF000015', 'EF000016', 'EF000017', 'EF000018', 'EF000019', 'EF000022', 'EF000024']

Preview rows:
    Articlenr  EF000001  EF000002  EF000003  EF000004  EF000005  EF000006  \
0   34581962       6.0       NaN       NaN       NaN       NaN       NaN   
1   35764300       NaN       NaN       NaN       NaN       NaN       NaN   
2   35823358       NaN       NaN       NaN       NaN       NaN       NaN   

   EF000007  EF000008  EF000010  ...  EFDE0031  EFDE0032  EFFR0001  EFFR0002  \
0       NaN       6.2       NaN  ...       NaN       NaN       NaN       NaN   
1       NaN       NaN       NaN  ...       NaN       NaN       NaN       NaN   
2       NaN       NaN       NaN  ...       NaN       NaN       NaN       NaN   

   EFNL0014  EFUK0017  EFUK0019  ETIM  statistiek_hoofdgroep  

In [5]:
assert "Articlenr" in features.columns, "Articlenr not found!"      
# ensuring that the key exists

features["articlenr"] = features["Articlenr"].astype("int64")       
# creating a lowercase merge key to match with the transactions files


In [6]:
rows_before = features.shape[0]                                   
  # remembering the original row count
features = features.drop_duplicates(subset=["Articlenr"])           
# keeping only the first row per unique product id
rows_after = features.shape[0]                                     
 # finding out the new row count

print(f"\nDuplicates removed (by Articlenr): {rows_before - rows_after}") 
 # reports how many were duplicates were removed



Duplicates removed (by Articlenr): 2711


In [7]:
for col in ["statistiek_hoofdgroep", "statistiek_subgroep"]:       
     # looping through the two categorical columns
    if col in features.columns:                                     # this code only acts if the column exists
        features[col] = features[col].astype("string")              # making it text 
        features[col] = features[col].str.strip()                   # removing any accidental leading spaces
        features[col] = features[col].astype("category")            # storing it as category as i saves memory

if "ETIM" in features.columns:                                      # ETIM is the numeric descriptor
    features["ETIM"] = pd.to_numeric(features["ETIM"],              # converting all to numbers and invalid data will become NaN
                                      errors="coerce")

In [8]:
missing_pct = features.isna().mean().sort_values(ascending=False)  
 # percentage missing per column 

print("\nTop-15 most-missing columns (%):")
print((missing_pct.head(15) * 100).round(1))                       
 # showing the most missed columns 

KEEP_COLS = {                                                       
     # columns not to drop because I need them for the anaylsis
    "Articlenr", "articlenr", "ETIM",
    "statistiek_hoofdgroep", "statistiek_subgroep"
}

THRESH = 0.80                                                       # drop columns with >80% missing (tunable)



Top-15 most-missing columns (%):
EF006634    100.0
EF012526    100.0
EF009515    100.0
EF002963    100.0
EF008203    100.0
EF012524    100.0
EF008875    100.0
EF006273    100.0
EF006781    100.0
EF008214    100.0
EF007342    100.0
EF007340    100.0
EF007344    100.0
EF003984    100.0
EF003312    100.0
dtype: float64


In [9]:
to_drop = []                                                       
  # collecting the columns to drop them in this list/place.

for col in features.columns:                                         # scaning all the columns
    if col in KEEP_COLS:                                             # skipping the essential columns
        continue
    if missing_pct.get(col, 0.0) > THRESH:                           # if more than 80% is missing then that column will be dropped.
        to_drop.append(col)                                          

In [10]:
print(f"\nColumns to drop (> {int(THRESH*100)}% missing):", len(to_drop)) 
 # report how many will go
print("Sample to drop:", to_drop[:10])                             
  # peek a few

features = features.drop(columns=to_drop)                            # drop them the not-needed/missing columns
print("Shape after dropping high-missing columns:", features.shape)  # new size of columns



Columns to drop (> 80% missing): 4016
Sample to drop: ['EF000001', 'EF000002', 'EF000003', 'EF000004', 'EF000005', 'EF000006', 'EF000012', 'EF000013', 'EF000014', 'EF000015']
Shape after dropping high-missing columns: (166228, 13)


In [11]:
const_cols = []                                                     
 # collecting the constant columns in this list/place.

for col in features.columns:                                         # checking each remaining column
    if col in KEEP_COLS:                                             # Do not drop the essential columns
        continue
    if features[col].nunique(dropna=True) <= 1:                      # 0 or 1 unique then its not needed.
        const_cols.append(col)

In [12]:
print("\nConstant/no-variance columns:", len(const_cols))            # Number of constants that were found
print("Sample constants:", const_cols[:10])                          # seeing a sample 

features = features.drop(columns=const_cols)                         # removing the constants
print("Shape after dropping constants:", features.shape)             # new size of the data




Constant/no-variance columns: 0
Sample constants: []
Shape after dropping constants: (166228, 13)


In [13]:
ef_cols = [c for c in features.columns if c.startswith("EF")]       
base_cols = [                                                     
       # the Important columns first
    "Articlenr", "articlenr", "ETIM",
    "statistiek_hoofdgroep", "statistiek_subgroep"
]

In [14]:
ordered_cols = [c for c in base_cols if c in features.columns] + ef_cols 
 # keeping existing ones 
features = features[ordered_cols]                                
   # putting it in order

print("\nFinal column order set. First 12 columns:")
print(features.columns[:12].tolist())                            
   # seeing a sample


Final column order set. First 12 columns:
['Articlenr', 'articlenr', 'ETIM', 'statistiek_hoofdgroep', 'statistiek_subgroep', 'EF000007', 'EF000008', 'EF000010', 'EF000040', 'EF000049', 'EF000139', 'EF002169']


In [15]:
if "ETIM" in features.columns:                                      # finding out the negatives and replacing them 
    neg_count = (features["ETIM"] < 0).sum(skipna=True)             
    if neg_count > 0:                                               
        print(f"\nETIM negatives found and set to NaN: {neg_count}")
        features.loc[features["ETIM"] < 0, "ETIM"] = np.nan         # replacig the negatives with NaN



In [16]:
features.to_parquet(CLEAN_FEATURES, index=False)                    
print("\nSaved cleaned features →", CLEAN_FEATURES.resolve())      
print("Final shape:", features.shape)                               
#making the new cleaned features file and saving it in the vs code folder.



Saved cleaned features → C:\Users\sreej\Downloads\stats\sub-assignments\SUB2\item_features_clean.parquet
Final shape: (166228, 13)


In [17]:
summary = {                                                         # just checking overall
    "rows": features.shape[0],                                      # number of products
    "cols": features.shape[1],                                      # number of columns after the cleaning
    "kept_main_cols": [c for c in base_cols if c in features.columns],  # what was kept
    "num_EF_cols": sum(c.startswith("EF") for c in features.columns)    # the count of the remaining EF features
}
print("\nCLEANING SUMMARY:", summary)                     


CLEANING SUMMARY: {'rows': 166228, 'cols': 13, 'kept_main_cols': ['Articlenr', 'articlenr', 'ETIM', 'statistiek_hoofdgroep', 'statistiek_subgroep'], 'num_EF_cols': 8}


**After cleaning the features dataset**

After completing all cleaning steps, the dataset is now much smaller and more usable.
Duplicate items were removed, categorical columns were cleaned, ETIM was converted to a proper numeric type, and EF-columns with too many missing values or no variance were dropped. Finally, the important columns were reordered and the cleaned file was saved.

Final output:

-Rows: 166,228

-Columns: 13

-Main columns kept: Articlenr, articlenr, ETIM, statistiek_hoofdgroep, statistiek_subgroep

-Remaining EF-feature columns: 8

**Cleaning the Transactions Dataset**

In this part, I loaded the raw transactions file and started preparing it for analysis.
First, I inspected the basic structure (rows, columns, and sample data), then converted all column names to lowercase and fixed their data types (integers, dates, numeric amounts, etc.).
The goal here was to standardise the dataset, remove duplicates, clean messy fields like channel, and check for invalid or missing values before using the data for ANOVA (anova for sub-question 2) and merging later.

In [18]:

import pandas as pd          # 'pd' is a short alias for the pandas library 
import numpy as np           # 'np' is a short alias for numpy
from pathlib import Path     # 'Path' helps build file paths in a safe/folder

In [19]:
# Creating a base path for the transcations file. Path(".") means "the current folder".
DATA_PATH = Path(".")


In [20]:
# Building full paths to the input/output files.
# The slash "/" is overloaded by Path to mean "join path parts".
RAW_TX = DATA_PATH / "TaoYin_User_Transactions_v2.parquet"
CLEAN_TX = DATA_PATH / "transactions_clean.parquet"


In [21]:

# pd.read_parquet reads the parquet file into the DataFrame.
tx = pd.read_parquet(RAW_TX)

In [22]:
# shape uses a dot "." to access an "attribute" (a stored property) of the DataFrame.
# It returns a (rows, columns) tuple. The comma "," separates items in the tuple.
#printing the number of rows and columns
print("Shape (rows, cols):", tx.shape)

# .columns returns all column names. [:20] slices the first 20 items.
print("\nFirst 20 columns:", list(tx.columns[:20]))

# .head(5) prints the first 5 rows.
print("\nPreview:")
print(tx.head(5))



Shape (rows, cols): (492731, 10)

First 20 columns: ['customer', 'invoice', 'invoice_date', 'articlenr', 'volume', 'sum1', 'sum2', 'postcode_delivery', 'postcode_customer', 'channel']

Preview:
   customer  invoice invoice_date  articlenr  volume     sum1     sum2  \
0      2284  1187467   2020-01-02   32879960      24   343.73   362.30   
1      2284  1187467   2020-01-02   32879962      60  1067.64  1119.96   
2      2284  1187467   2020-01-02   32841810     120  2214.00  2408.40   
3      1704  1191447   2020-01-17   36866802       1    21.52    19.32   
4        45  1187691   2020-01-03   34360272       4    28.91    28.49   

   postcode_delivery  postcode_customer    channel  
0              80413              80413  Tel 86041  
1              80413              80413  Tel 86041  
2              80413              80413  Tel 86041  
3              81724              81724  OTC 81812  
4              81796              81796  Web Order  


In [23]:
# making a copy so i don't accidentally overwrite the original DataFrame in the memory.
tx = tx.copy()

# Converting column names to lowercase for consistency.
# .columns accesses the list-like Index of names; .str.lower() converting each name to lower-case.
tx.columns = tx.columns.str.lower()

In [24]:
# expected list the columns.
expected = [
    "customer", "invoice", "invoice_date", "articlenr", "volume",
    "sum1", "sum2", "postcode_delivery", "postcode_customer", "channel"
]
print("\nAll expected columns present?:", set(expected).issubset(set(tx.columns)))


All expected columns present?: True


In [25]:
# Ensuring that all the key columns have the correct data types.
# .astype("int64") converts the series to 64-bit integer.
tx["customer"] = tx["customer"].astype("int64")
tx["invoice"] = tx["invoice"].astype("int64")
tx["articlenr"] = tx["articlenr"].astype("int64")

In [26]:
# Parse dates. pd.to_datetime(...) converts text to datetime objects.
tx["invoice_date"] = pd.to_datetime(tx["invoice_date"], errors="coerce")


In [27]:
# Numeric money-type fields: coerce invalid strings to NaN (missing).
tx["sum1"] = pd.to_numeric(tx["sum1"], errors="coerce")
tx["sum2"] = pd.to_numeric(tx["sum2"], errors="coerce")

In [28]:
# volume is quantity; making it an integer if it is in whole numbers. If not guaranteed, keeping it as a float.
tx["volume"] = pd.to_numeric(tx["volume"], errors="coerce")

In [29]:
# Postcodes are codes; they seem numeric in file/data. Keep as int64 if not missing.
# If there are  missing values, using Int64 (nullable integer) instead of int64.
tx["postcode_delivery"]  = pd.to_numeric(tx["postcode_delivery"], errors="coerce").astype("Int64")
tx["postcode_customer"]  = pd.to_numeric(tx["postcode_customer"], errors="coerce").astype("Int64")


In [30]:
# Channels are text; normalize the whitespace and keeping it as a string (pandas' string dtype).
tx["channel"] = tx["channel"].astype("string").str.strip()


In [31]:
# Counting rows before dropping the duplicates.
before = tx.shape[0]

In [32]:
# .drop_duplicates() removes rows that are exactly the same across all columns.
# subset=[...] restricts the comparison to specific columns (optional).
# keep="first" keeps the first occurrence and drops later duplicates.
tx = tx.drop_duplicates(keep="first")

after = tx.shape[0]
print(f"\nDuplicate rows removed: {before - after}")




Duplicate rows removed: 17534


In [33]:
# Dropping rows with missing essential fields:
# .dropna(subset=[...]) removes rows if any of those columns have NaN.
tx = tx.dropna(subset=["invoice_date", "articlenr", "volume", "sum2", "channel"])

# Removing any non-sensical values:
# volume should be more than 0 (no negative or zero quantities)
tx = tx[tx["volume"] > 0]

# sum2 is the final amount; it should be more than 0
tx = tx[tx["sum2"] > 0]

print("After validity filters, shape:", tx.shape)

After validity filters, shape: (475116, 10)


In [34]:
# In the transcation file, channel has values like "Tel 86041", "OTC 81812", "Web Order".
# Making a clean label: the first word (letters only) at the start.
# .str.extract(r'(^[A-Za-z]+)') uses a regular expression (regex).
# r'...' is a raw string so backslashes are not treated specially by Python itself.
tx["channel_clean"] = tx["channel"].str.extract(r'(^[A-Za-z]+)')

# Some "Web Order" may become "Web". 
# The curly braces { } create a dictionary: "key": "value", entries separated by commas.
mapping = {
    "Web": "Web",
    "Tel": "Tel",
    "OTC": "OTC"
}
# .map(...) replaces values using the mapping; .fillna(...) fills any non-mapped with the original clean value.
tx["channel_clean"] = tx["channel_clean"].map(mapping).fillna(tx["channel_clean"])

# Making it a categorical variable (good for grouping and saves on memory).
tx["channel_clean"] = tx["channel_clean"].astype("category")

print("\nChannel counts:")
print(tx["channel_clean"].value_counts(dropna=False))



Channel counts:
channel_clean
Tel          144586
ERP          127858
Web          123713
OTC           74888
MobileApp      4052
Others           19
Name: count, dtype: int64


In [35]:
# For ANOVA on 'sum2', extreme outliers can dominate.
# So I created a winsorized version: capping it at top 1% at the 99th percentile.
# .quantile(0.99) returns the 99th percentile.
p99 = tx["sum2"].quantile(0.99)

# .clip(upper=p99) caps values above p99 to p99.
tx["sum2_winsor"] = tx["sum2"].clip(upper=p99)

print("\nWinsorization cap (99th percentile) for sum2:", float(p99))



Winsorization cap (99th percentile) for sum2: 1506.8644999999915


In [36]:
# .describe() gives summary stats; .groupby(...).size() counts per group.
print("\nSummary stats (sum2 and sum2_winsor):")
print(tx[["sum2", "sum2_winsor"]].describe())



Summary stats (sum2 and sum2_winsor):
               sum2    sum2_winsor
count  4.751160e+05  475116.000000
mean   2.656857e+02     113.698032
std    8.180267e+04     220.732958
min    1.000000e-02       0.010000
25%    1.247000e+01      12.470000
50%    3.720000e+01      37.200000
75%    1.108800e+02     110.880000
max    5.599838e+07    1506.864500


In [37]:
print("\nCounts by channel_clean:")
print(tx.groupby("channel_clean").size())

# Checking the date range to ensure the period looks right.
print("\nInvoice date range:", tx["invoice_date"].min(), "→", tx["invoice_date"].max())



Counts by channel_clean:
channel_clean
ERP          127858
MobileApp      4052
OTC           74888
Others           19
Tel          144586
Web          123713
dtype: int64

Invoice date range: 2020-01-02 00:00:00 → 2024-09-25 00:00:00


C:\Users\sreej\AppData\Local\Temp\ipykernel_4228\1989141854.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(tx.groupby("channel_clean").size())


In [38]:
# Saving the cleaned file as a parquet (fast, keeps dtypes). index=False prevents writing row numbers as a column.
tx.to_parquet(CLEAN_TX, index=False)

print("\nSaved cleaned transactions to:", CLEAN_TX.resolve())
print("Final shape:", tx.shape)



Saved cleaned transactions to: C:\Users\sreej\Downloads\stats\sub-assignments\SUB2\transactions_clean.parquet
Final shape: (475116, 12)


**Final Cleaning Summary**

After applying all cleaning steps, the transactions dataset is now fully standardised and filtered. Duplicate rows were removed, incorrect or missing values were dropped, and the channel field was cleaned into a usable category. A winsorized version of sum2 was also created to reduce the influence of extreme outliers for ANOVA.
Finally, the cleaned dataset was saved as a new parquet file, ready for merging with the item features and for further statistical analysis.